In [ ]:
# Import thư viện và khởi tạo
import ee
import geemap
import os
import numpy as np
from datetime import datetime

# Khởi tạo Google Earth Engine với project ID
ee.Initialize(project='ee-bonglantrungmuoi')

# Cấu hình thời gian và tham số
start_date = '2022-01-01'
end_date = '2025-01-01'
CLOUD_COVER_THRESHOLD = 20  # Ngưỡng mây tối đa (%)
PROCESSING_LEVEL = 'HARMONIZED'  # Sử dụng Harmonized collection

# Tạo thư mục lưu dữ liệu
output_folder = './sentinel2_data'
os.makedirs(output_folder, exist_ok=True)

print(f"=== CẤU HÌNH TẢI SENTINEL-2 ===")
print(f"- Thời gian: {start_date} đến {end_date}")
print(f"- Ngưỡng mây: {CLOUD_COVER_THRESHOLD}%")
print(f"- Processing level: {PROCESSING_LEVEL}")
print(f"- Thư mục lưu: {output_folder}")
print(f"- Sử dụng: geemap cho file lớn")
print(f"- Mục tiêu: Dữ liệu quang học cho phân tích độ ẩm đất")

In [ ]:
# Đọc shapefile và tạo Sentinel-2 collection
print("=== TẢI SHAPEFILE VÀ TẠO COLLECTION ===")

# Sử dụng shapefile Lâm Đồng từ GEE Asset
lamdong_asset = ee.FeatureCollection('projects/ee-bonglantrungmuoi/assets/lamdong')
lamdong_geometry = lamdong_asset.geometry()

# Hiển thị thông tin về shapefile
print(f"✓ Đã tải shapefile từ GEE Asset")
print(f"  - Số features: {lamdong_asset.size().getInfo()}")

# Tạo Sentinel-2 collection (SR - Surface Reflectance)
if PROCESSING_LEVEL == 'HARMONIZED':
    collection_name = 'COPERNICUS/S2_SR_HARMONIZED'
else:
    collection_name = 'COPERNICUS/S2_SR'

sentinel2_collection = (ee.ImageCollection(collection_name)
                        .filterBounds(lamdong_geometry)
                        .filterDate(start_date, end_date)
                        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_COVER_THRESHOLD))
                        .sort('system:time_start'))

# Thông tin collection
collection_size = sentinel2_collection.size().getInfo()
print(f"✓ Đã tạo Sentinel-2 collection")
print(f"  - Dataset: {collection_name}")
print(f"  - Tổng số ảnh: {collection_size}")

# Thông tin ảnh đầu tiên
if collection_size > 0:
    first_image = ee.Image(sentinel2_collection.first())
    first_date = ee.Date(first_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    first_cloud = first_image.get('CLOUDY_PIXEL_PERCENTAGE').getInfo()
    print(f"  - Ảnh đầu tiên: {first_date} (mây: {first_cloud:.1f}%)")
    
    # Thống kê theo năm
    for year in range(2022, 2025):
        year_collection = sentinel2_collection.filter(ee.Filter.calendarRange(year, year, 'year'))
        year_count = year_collection.size().getInfo()
        
        if year_count > 0:
            # Tính mây trung bình
            avg_cloud = year_collection.aggregate_mean('CLOUDY_PIXEL_PERCENTAGE').getInfo()
            print(f"  - Năm {year}: {year_count} ảnh (mây TB: {avg_cloud:.1f}%)")
        else:
            print(f"  - Năm {year}: 0 ảnh")
else:
    print("⚠️ Không tìm thấy ảnh nào!")

# Hiển thị bands có sẵn
if collection_size > 0:
    band_names = first_image.bandNames().getInfo()
    print(f"\n📊 Bands có sẵn ({len(band_names)}):")
    for i, band in enumerate(band_names):
        print(f"  {i+1:2d}. {band}")
    
    # Bands quan trọng cho phân tích độ ẩm đất
    important_bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']  # Blue, Green, Red, NIR, SWIR1, SWIR2
    print(f"\n💡 Bands quan trọng cho độ ẩm đất: {', '.join(important_bands)}")

In [ ]:
# Functions xử lý Sentinel-2 - ĐƠN GIẢN
def mask_clouds_s2(image):
    """Loại bỏ mây và shadow"""
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask)

def add_indices(image):
    """Tính toán các chỉ số"""
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    lswi = image.normalizedDifference(['B8', 'B12']).rename('LSWI')
    savi = image.expression('((NIR - RED) / (NIR + RED + L)) * (1 + L)', {
        'NIR': image.select('B8'), 'RED': image.select('B4'), 'L': 0.5
    }).rename('SAVI')
    return image.addBands([ndvi, ndwi, lswi, savi])

def download_monthly_composite(year, month, resolution=10):
    """Tải composite tháng"""
    # Tạo tên file
    filename = f'S2_Lamdong_{year}_{month:02d}_res{resolution}m_indices.tif'
    filepath = os.path.join(output_folder, filename)
    
    # Kiểm tra file đã tồn tại
    if os.path.exists(filepath):
        file_size = os.path.getsize(filepath) / (1024*1024)
        print(f"⏭️  Skip {filename} ({file_size:.1f} MB)")
        return filepath
    
    # Tạo collection tháng
    start_month = ee.Date.fromYMD(year, month, 1)
    end_month = start_month.advance(1, 'month')
    monthly_collection = sentinel2_collection.filterDate(start_month, end_month)
    
    # Kiểm tra có dữ liệu
    count = monthly_collection.size().getInfo()
    if count == 0:
        print(f"❌ No data for {year}-{month:02d}")
        return None
    
    # Xử lý và tạo composite
    composite = (monthly_collection
                .map(mask_clouds_s2)
                .map(add_indices)
                .select(['NDVI', 'NDWI', 'LSWI', 'SAVI'])
                .median()
                .clip(lamdong_geometry))
    
    # Tải file
    try:
        print(f"? Downloading {filename}...")
        geemap.download_ee_image(
            image=composite,
            filename=filepath,
            scale=resolution,
            region=lamdong_geometry,
            crs='EPSG:4326'
        )
        
        if os.path.exists(filepath):
            file_size = os.path.getsize(filepath) / (1024*1024)
            print(f"✅ Saved {filename} ({file_size:.1f} MB)")
            return filepath
        else:
            print(f"❌ Failed {filename}")
            return None
            
    except Exception as e:
        print(f"❌ Error {year}-{month:02d}: {str(e)}")
        return None

def quick_check():
    """Kiểm tra nhanh"""
    if not os.path.exists(output_folder):
        print("📂 No data folder")
        return
    
    files = [f for f in os.listdir(output_folder) if f.endswith('.tif')]
    if not files:
        print("📂 No files")
        return
    
    total_size = sum(os.path.getsize(os.path.join(output_folder, f)) for f in files) / (1024*1024)
    print(f"📊 Downloaded: {len(files)}/36 files ({total_size:.1f} MB)")

print("✅ Functions ready")

In [ ]:
# TẢI MONTHLY COMPOSITE (2022-2024)
print("=== DOWNLOADING MONTHLY COMPOSITE ===")

# Kiểm tra hiện tại
quick_check()

# Tải 36 tháng
downloaded = 0
failed = 0

for year in range(2022, 2025):
    for month in range(1, 13):
        result = download_monthly_composite(year, month, 10)
        if result:
            downloaded += 1
        else:
            failed += 1

print(f"\n📊 SUMMARY:")
print(f"✅ Downloaded: {downloaded}")
print(f"❌ Failed: {failed}")

# Kiểm tra cuối
quick_check()

In [ ]:
# TẢI MONTHLY COMPOSITE (RESUME MODE)
print("=== RESUME DOWNLOAD MODE ===")

# Kiểm tra file thiếu
missing_months = []
for year in range(2022, 2025):
    for month in range(1, 13):
        filename = f'S2_Lamdong_{year}_{month:02d}_res10m_indices.tif'
        filepath = os.path.join(output_folder, filename)
        if not os.path.exists(filepath):
            missing_months.append((year, month))

if not missing_months:
    print("🎉 All 36 months completed!")
    quick_check()
else:
    print(f"📥 Need to download {len(missing_months)} months")
    
    # Tải các tháng thiếu
    for i, (year, month) in enumerate(missing_months, 1):
        print(f"[{i}/{len(missing_months)}] {year}-{month:02d}")
        download_monthly_composite(year, month, 10)
    
    print(f"\n✅ Resume download completed")
    quick_check()

In [ ]:
# KIỂM TRA KẾT QUẢ
print("=== FINAL CHECK ===")

if not os.path.exists(output_folder):
    print("❌ No output folder")
else:
    files = [f for f in os.listdir(output_folder) if f.endswith('.tif')]
    
    if not files:
        print("📂 No files downloaded")
    else:
        # Phân tích theo năm
        coverage = {}
        for file in files:
            if 'S2_Lamdong' in file:
                parts = file.split('_')
                if len(parts) >= 4:
                    year = int(parts[2])
                    month = int(parts[3])
                    if year not in coverage:
                        coverage[year] = []
                    coverage[year].append(month)
        
        total_size = sum(os.path.getsize(os.path.join(output_folder, f)) for f in files) / (1024*1024)
        
        print(f"📊 RESULTS:")
        print(f"Total files: {len(files)}")
        print(f"Total size: {total_size:.1f} MB ({total_size/1024:.1f} GB)")
        
        for year in sorted(coverage.keys()):
            months = sorted(coverage[year])
            print(f"{year}: {len(months)} months")
        
        total_months = sum(len(months) for months in coverage.values())
        print(f"Progress: {total_months}/36 months ({total_months/36*100:.1f}%)")
        
        if total_months == 36:
            print("🎉 All monthly composites completed!")
        else:
            missing = 36 - total_months
            print(f"⚠️  Still missing {missing} months")

print(f"\n? Data contains: NDVI, NDWI, LSWI, SAVI indices")
print(f"📁 Location: {output_folder}")
print(f"🎯 Ready for soil moisture analysis")